# Serving Machine Learning models with Google Vertex AI

Great to have you here, this is the code for the following article:

* https://medium.com/google-cloud/serving-machine-learning-models-with-google-vertex-ai-5d9644ededa3

Your feedback and questions are highly appreciated. <br>You can find me on Twitter [@HeyerSascha](https://twitter.com/HeyerSascha) or connect with me via [LinkedIn](https://www.linkedin.com/in/saschaheyer/). <br>Even better, subscribe to my [YouTube](https://www.youtube.com/channel/UC--Sm3D-rqCUeLXmraypdPQ) channel ❤️.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
! gcloud config set project sascha-playground-doit

Updated property [core/project].


## Custom Prediction Container with FastAPI

In [ ]:
%%writefile main.py
import uvicorn

import tensorflow as tf
import os
import numpy as np
from enum import Enum
from typing import List, Optional
from pydantic import BaseModel

from fastapi import Request, FastAPI, Response
from fastapi.responses import JSONResponse
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = TFDistilBertForSequenceClassification.from_pretrained("../sentiment")

app = FastAPI(title="Sentiment Analysis")

AIP_HEALTH_ROUTE = os.environ.get('AIP_HEALTH_ROUTE', '/health')
AIP_PREDICT_ROUTE = os.environ.get('AIP_PREDICT_ROUTE', '/predict')

class Prediction(BaseModel):
  sentiment: str 
  confidence: Optional[float]

class Predictions(BaseModel):
    predictions: List[Prediction]

# instad of creating a class we could have also loaded this information
# from the model configuration. Better if you introduce new labels over time
class Sentiment(Enum):
  NEGATIVE = 0
  POSITIVE = 1


@app.get(AIP_HEALTH_ROUTE, status_code=200)
async def health():
    return {'health': 'ok'}

@app.post(AIP_PREDICT_ROUTE, 
          response_model=Predictions,
          response_model_exclude_unset=True)
async def predict(request: Request):
    body = await request.json()
    print(body)

    instances = body["instances"]
    print(instances)
    print(type(instances))
    instances = [x['text'] for x in instances]
    print(instances)

    tf_batch = tokenizer(instances, max_length=128, padding=True,
                            truncation=True, return_tensors='tf')

    print(tf_batch)

    tf_outputs = model(tf_batch)

    print(tf_outputs)

    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    print(tf_predictions)

    indices = np.argmax(tf_predictions, axis=-1)
    confidences = np.max(tf_predictions, axis=-1)

    outputs = []

    for index, confidence in zip(indices, confidences):
      sentiment = Sentiment(index).name
      print(index)
      print(confidence)
      outputs.append(Prediction(sentiment=sentiment, confidence=confidence))

    return Predictions(predictions=outputs)

if __name__ == "__main__":
  app.run(debug=True, host="0.0.0.0",port=8080)

Overwriting main.py


In [ ]:
%%writefile Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.8-slim
RUN pip install --no-cache-dir transformers==4.1.1 tensorflow==2.9.1 numpy==1.23.1 pydantic==1.9.1
COPY main.py ./main.py
COPY ./sentiment /sentiment

Overwriting Dockerfile


In [ ]:
%%writefile cloudbuild.yaml
steps:
# Download the model to embed it into the image
- name: 'gcr.io/cloud-builders/gsutil'
  args: ['cp','-r', 'gs://doit-vertex-demo/models/sentiment', '.']
  id: 'download-model'
# Build the container image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/sascha-playground-doit/sentiment-fast-api', '.']
  waitFor: ['download-model']
# Push the container image to Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'gcr.io/sascha-playground-doit/sentiment-fast-api']

images:
- gcr.io/sascha-playground-doit/sentiment-fast-api

Writing cloudbuild.yaml


In [ ]:
!gcloud builds submit --config cloudbuild.yaml

Creating temporary tarball archive of 25 file(s) totalling 54.2 MiB before compression.
Uploading tarball of [.] to [gs://sascha-playground-doit_cloudbuild/source/1667389674.261614-ea5c988004d04af3b1e5504d1d35cd0d.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/sascha-playground-doit/locations/global/builds/75c0fb6f-a11b-4552-926f-d6a5aab12ab2].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/75c0fb6f-a11b-4552-926f-d6a5aab12ab2?project=234439745674 ].
 REMOTE BUILD OUTPUT
starting build "75c0fb6f-a11b-4552-926f-d6a5aab12ab2"

FETCHSOURCE
Fetching storage object: gs://sascha-playground-doit_cloudbuild/source/1667389674.261614-ea5c988004d04af3b1e5504d1d35cd0d.tgz#1667389685171779
Copying gs://sascha-playground-doit_cloudbuild/source/1667389674.261614-ea5c988004d04af3b1e5504d1d35cd0d.tgz#1667389685171779...
/ [1 files][  6.5 MiB/  6.5 MiB]                                                
Operation completed over 1 objects/6.5 MiB.
BUILD
Starting Ste

## Upload and deploy model

In [ ]:
!gcloud ai models upload \
  --container-ports=80 \
  --container-predict-route="/predict" \
  --container-health-route="/health" \
  --region=us-central1 \
  --display-name=sentiment-fast-api \
  --container-image-uri=gcr.io/sascha-playground-doit/sentiment-fast-api

Using endpoint [https://us-central1-aiplatform.googleapis.com/]


In [ ]:
!gcloud ai endpoints create \
  --project=sascha-playground-doit \
  --region=us-central1 \
  --display-name=sentiment-fast-api

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Created Vertex AI endpoint: projects/234439745674/locations/us-central1/endpoints/7608484124768075776.


get model and endpoint IDs from previous steps
deployment takes 15 min aprox

In [ ]:
!gcloud ai endpoints deploy-model 7608484124768075776 \
  --project=sascha-playground-doit \
  --region=us-central1 \
  --model=8709323962590429184 \
  --traffic-split=0=100 \
  --machine-type="n1-standard-2" \
  --display-name=sentiment

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Deployed a model to the endpoint 7608484124768075776. Id of the deployed model: 1352896281420234752.


## Predictions


### gcloud

In [ ]:
%%writefile request.json
{"instances": [
  {"text":"DoiT is a great company."},
  {"text":"The beach was nice but overall the hotel was very bad."}
]}

Overwriting request.json


In [ ]:
!gcloud ai endpoints predict 7608484124768075776 \
  --region=us-central1 \
  --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'confidence': 0.9409326314926147, 'sentiment': 'POSITIVE'}, {'confidence': 0.9964427351951599, 'sentiment': 'NEGATIVE'}]


### Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform==1.14.0 --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.8 MB/s 
     |████████████████████████████████| 1.0 MB 63.0 MB/s 
     |████████████████████████████████| 233 kB 62.8 MB/s 
     |████████████████████████████████| 47 kB 4.6 MB/s 
     |████████████████████████████████| 106 kB 40.2 MB/s 
     |████████████████████████████████| 233 kB 41.9 MB/s 
     |████████████████████████████████| 231 kB 45.6 MB/s 
     |████████████████████████████████| 231 kB 47.2 MB/s 
     |████████████████████████████████| 230 kB 46.2 MB/s 
     |████████████████████████████████| 106 kB 46.1 MB/s 
     |████████████████████████████████| 107 kB 45.3 MB/s 
     |████████████████████████████████| 107 kB 45.8 MB/s 
     |████████████████████████████████| 107 kB 45.5 MB/s 
     |████████████████████████████████| 106 kB 48.7 MB/s 
     |████████████████████████████████| 106 kB 59.0 MB/s 
     |████████████████████████████

In [ ]:
from google.cloud import aiplatform

project = 'sascha-playground-doit'
location = 'us-central1'

aiplatform.init(project=project,
                location=location)

In [ ]:
instances = [
      {
         "text":"DoiT is a great company."
      },
      {
         "text":"The beach was nice but overall the hotel was very bad."
      }
   ]


endpoint = aiplatform.Endpoint("projects/234439745674/locations/us-central1/endpoints/7608484124768075776")

prediction = endpoint.predict(instances=instances)
print(prediction)

Prediction(predictions=[{'sentiment': 'POSITIVE', 'confidence': 0.9409326314926147}, {'confidence': 0.9964427351951599, 'sentiment': 'NEGATIVE'}], deployed_model_id='1352896281420234752', explanations=None)
